In [2]:
pip install inflect


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
import re
import inflect

def clean_column_headers(headers):
    """Cleans column headers by normalizing, removing unwanted characters, and singularizing words."""
    p = inflect.engine()
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            # Convert to lowercase
            header = header.lower()
            # Normalize whitespace
            header = re.sub(r'\s+', ' ', header)
            # Remove contents in parentheses
            header = re.sub(r'\(.*?\)', '', header)
            # Remove unwanted characters
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            # Trim leading/trailing whitespace
            header = header.strip()
            
            # Singularize each word using inflect
            words = header.split()
            singular_words = [p.singular_noun(word) if p.singular_noun(word) else word for word in words]
            header = " ".join(singular_words)
            
        cleaned_headers.append(header)
    return cleaned_headers

# Example usage:
headers = ["Upgrades (New)", "Sales", "Discounts", "Product IDs", "Type of Upgrade", "Estimated Cost x 1,000 Constant Dollars (2012)"]
print(clean_column_headers(headers))


['upgrade', 'sale', 'discount', 'product id', 'type of upgrade', 'estimated cost x 1000 constant dollar']


In [92]:
import os
import pdfplumber
import pandas as pd
import re
import PyPDF2
import traceback
import inflect

# Define paths and project range
BASE_DIRECTORY = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/03_data"
OUTPUT_CSV_PATH_ORIGINAL = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/03_raw/rawdata_cluster5_style_D_originals.csv"
OUTPUT_CSV_PATH_ADDENDUM = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/03_raw/rawdata_cluster5_style_D_addendums.csv"
LOG_FILE_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/03_raw/scraping_cluster5_style_D_log.txt"
PROJECT_RANGE = range(869, 943)  # Example range for q_ids in Clusters 5

# Initialize DataFrames
core_originals = pd.DataFrame()
core_addendums = pd.DataFrame()

# Initialize tracking variables
scraped_projects = set()
skipped_projects = set()
missing_projects = set()
scraped_pdfs = []
skipped_pdfs = []
addendum_pdfs = []
original_pdfs = []
total_pdfs_accessed = 0
total_pdfs_scraped = 0
total_pdfs_skipped = 0

def clean_column_headers(headers):
    """Cleans column headers by normalizing, removing unwanted characters, and singularizing words."""
    p = inflect.engine()
    cleaned_headers = []
    for header in headers:
        if header is None:
            header = ""
        elif isinstance(header, str):
            header = header.lower()
            header = re.sub(r'\s+', ' ', header)
            header = re.sub(r'\(.*?\)', '', header)
            header = re.sub(r'[^a-zA-Z0-9\s]', '', header)
            header = header.strip()
            # Correct any mis‐spellings of “type of upgrade”
            header = re.sub(r'\btype of upgr\s*ade\b', 'type of upgrade', header)
            words = header.split()
            singular_words = [p.singular_noun(word) if p.singular_noun(word) else word for word in words]
            header = " ".join(singular_words)
        cleaned_headers.append(header)
    return cleaned_headers

def clean_string_cell(value):
    """Cleans string cells by removing newlines and trimming spaces."""
    if isinstance(value, str):
        return value.replace('\n', ' ').strip()
    return value

def contains_phrase(row, phrase):
    """Checks if any cell in a row contains a specific phrase."""
    regex_pattern = re.sub(r"\s+", r"\\s*", phrase)
    pattern = re.compile(regex_pattern, flags=re.IGNORECASE)
    return row.astype(str).apply(lambda cell: bool(pattern.search(cell))).any()

def extract_specific_phrase(title):
    """
    Extracts a specific phrase from the table title based on predefined keywords.
    """
    phrases = [
        "PTO",
        "Reliability Network Upgrade",
        "Area Delivery Network Upgrade",
        "Local Delivery Network",
        "ADNU",
        "LDNU",
        "RNU"
    ]
    for phrase in phrases:
        if re.search(rf"\b{re.escape(phrase)}\b", title, re.IGNORECASE):
            return phrase
    return title  # Fallback if none found

def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type of upgrade",
        "upgrade",
        "description",
        "cost allocation factor"
    ]
    existing_desired = [col for col in desired_order if col in df.columns]
    remaining = [col for col in df.columns if col not in existing_desired]
    new_order = existing_desired + remaining
    df = df[new_order]
    return df

def search_gps_coordinates(text, log_file):
    """Search for GPS coordinates using multiple patterns."""
    gps_coords = re.search(r"gps coordinates:\s*([\d\.\-]+),\s*([\d\.\-]+)", text, re.IGNORECASE)
    if gps_coords:
        print(f"Found GPS coordinates: {gps_coords.groups()}", file=log_file)
        return gps_coords.groups()
    project_coords = re.search(r"latitude[:\s]*([\d\.\-]+)[^\d]+longitude[:\s]*([\d\.\-]+)", text, re.IGNORECASE)
    if project_coords:
        print(f"Found project coordinates: {project_coords.groups()}", file=log_file)
        return project_coords.groups()
    gps_coords_directional = re.search(
        r"gps coordinates:\s*([\d\.\-]+)\s*[nNsS],\s*([\d\.\-]+)\s*[eEwW]", text, re.IGNORECASE)
    if gps_coords_directional:
        lat, lon = gps_coords_directional.groups()
        latitude = lat if "N" in text.upper() else f"-{lat}"
        longitude = lon if "E" in text.upper() else f"-{lon}"
        print(f"Found directional GPS coordinates: {(latitude, longitude)}", file=log_file)
        return (latitude, longitude)
    print("GPS coordinates not found.", file=log_file)
    return (None, None)

def adjust_rows_length(data_rows, headers):
    """
    Ensures each row in data_rows has exactly len(headers) columns.
    If a row is too short, it is padded with empty strings.
    If too long, it is truncated.
    """
    col_count = len(headers)
    for i in range(len(data_rows)):
        row = data_rows[i]
        if len(row) > col_count:
            data_rows[i] = row[:col_count]
        elif len(row) < col_count:
            data_rows[i].extend([""] * (col_count - len(row)))

def extract_table2(pdf_path, log_file):
    """
    Extracts the Point of Interconnection from Table 1 in the provided PDF.
    """
    print(f"\nProcessing {pdf_path} for Table 1 extraction...", file=log_file)
    point_of_interconnection = None
    poi_pattern = re.compile(r"Point\s+of\s+Interconnection", re.IGNORECASE)
    table_settings_list = [
        {"horizontal_strategy": "text", "vertical_strategy": "lines", "snap_tolerance": 1},
        {"horizontal_strategy": "lines", "vertical_strategy": "lines", "snap_tolerance": 2},
    ]
    try:
        with pdfplumber.open(pdf_path) as pdf:
            table1_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*(?:2|B\.1)\b", text, re.IGNORECASE):
                    table1_pages.append(i)
            if not table1_pages:
                print("No Table 2 found in the PDF.", file=log_file)
                return None
            first_page = table1_pages[0]
            last_page = table1_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 1
            print(f"Table 2 starts on page {scrape_start + 1} and ends on page {scrape_end + 1}", file=log_file)
            extraction_successful = False
            for page_number in range(scrape_start, min(scrape_end + 1, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1} for Table 2...", file=log_file)
                for attempt, table_settings in enumerate(table_settings_list, start=1):
                    print(f"\nAttempt {attempt} with table settings: {table_settings}", file=log_file)
                    tables = page.find_tables(table_settings=table_settings)
                    print(f"Found {len(tables)} table(s) on page {page_number + 1} with current settings.", file=log_file)
                    for table_index, table in enumerate(tables, start=1):
                        tab = table.extract()
                        if not tab:
                            print(f"Table {table_index} on page {page_number + 1} is empty. Skipping.", file=log_file)
                            continue
                        print(f"\n--- Table {table_index} on Page {page_number + 1} ---", file=log_file)
                        for row_num, row in enumerate(tab, start=1):
                            print(f"Row {row_num}: {row}", file=log_file)
                        for row_index, row in enumerate(tab, start=1):
                            for cell_index, cell in enumerate(row, start=1):
                                if cell and poi_pattern.search(cell):
                                    poi_col_index = cell_index
                                    adjacent_col_index = poi_col_index + 1
                                    if adjacent_col_index <= len(row):
                                        poi_value = clean_string_cell(row[adjacent_col_index - 1])
                                        if poi_value:
                                            point_of_interconnection = poi_value
                                            print(f"\nFound Point of Interconnection: '{point_of_interconnection}' (Page {page_number + 1}, Table {table_index}, Row {row_index})", file=log_file)
                                            extraction_successful = True
                                            break
                                        else:
                                            print(f"\nPoint of Interconnection label found but adjacent value is empty (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                            poi_value_parts = []
                                            current_row_idx = row_index - 1
                                            start_scan = max(0, current_row_idx - 2)
                                            end_scan = min(len(tab), current_row_idx + 3)
                                            print(f"Scanning rows {start_scan + 1} to {end_scan} for POI value parts.", file=log_file)
                                            for scan_row_index in range(start_scan, end_scan):
                                                if scan_row_index == current_row_idx:
                                                    continue
                                                scan_row = tab[scan_row_index]
                                                if adjacent_col_index - 1 < len(scan_row):
                                                    scan_cell = clean_string_cell(scan_row[adjacent_col_index - 1])
                                                    if scan_cell and not poi_pattern.search(scan_cell):
                                                        poi_value_parts.append(scan_cell)
                                                        print(f"Found POI part in row {scan_row_index + 1}: '{scan_cell}'", file=log_file)
                                                    elif poi_pattern.search(scan_cell):
                                                        print(f"Encountered another POI label in row {scan_row_index + 1}. Skipping this row.", file=log_file)
                                                        continue
                                            if poi_value_parts:
                                                point_of_interconnection = " ".join(poi_value_parts)
                                                print(f"\nConcatenated Point of Interconnection: '{point_of_interconnection}' (Page {page_number + 1}, Table {table_index})", file=log_file)
                                                extraction_successful = True
                                                break
                                            else:
                                                print(f"\nNo POI value found in the surrounding rows (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                                    else:
                                        print(f"\nPoint of Interconnection label found but no adjacent column (Page {page_number + 1}, Table {table_index}, Row {row_index}).", file=log_file)
                            if extraction_successful:
                                break
                        if extraction_successful:
                            break
                    if extraction_successful:
                        break
                if extraction_successful:
                    break
    except Exception as e:
        print(f"Error processing Table 2 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return None
    if not extraction_successful:
        if point_of_interconnection is not None and point_of_interconnection != "":
            print("Point of Interconnection label found but no adjacent value.", file=log_file)
            return "Value Missing"
        else:
            print("Point of Interconnection not found in Table 2.", file=log_file)
            return None
    return point_of_interconnection

def fix_column_names(columns):
    """
    Renames duplicate and empty column names.
    Duplicate names are suffixed with _1, _2, etc.
    Empty or whitespace-only names are replaced with unnamed_1, unnamed_2, etc.
    """
    new_cols = []
    counts = {}
    unnamed_count = 1
    for col in columns:
        # Treat empty or whitespace-only names as unnamed.
        if not col or col.strip() == "":
            new_col = f"unnamed_{unnamed_count}"
            unnamed_count += 1
        else:
            new_col = col.strip()
        if new_col in counts:
            new_col_with_suffix = f"{new_col}_{counts[new_col]}"
            counts[new_col] += 1
            new_cols.append(new_col_with_suffix)
        else:
            counts[new_col] = 1
            new_cols.append(new_col)
    return new_cols

def post_process_columns(df, log_file):
    """
    Post-processes DataFrame column names:
      1. For any column named 'unnamed_#' (or empty), look at its first non-empty cell.
         If that cell is not a dollar amount (i.e. does not match /^\$\s*\d+(?:,\d{3})*(?:\.\d+)?$/)
         and it contains 2 or 3 words, then rename the column to that value (after cleaning).
         If a column already exists with that name, merge the data from the renamed column into the
         existing column and drop the renamed column.
      2. If a column is named "Needed For", then rename it to "description" (merging with an existing
         description column if necessary).
    """
    # Process unnamed columns.
    for col in list(df.columns):
        if col.lower().startswith("unnamed_") or col.strip() == "":
            # Find the first non-empty cell in this column.
            first_non_empty = None
            for val in df[col]:
                cell_val = ""
                if isinstance(val, str):
                    cell_val = val.strip()
                elif val is not None:
                    cell_val = str(val).strip()
                if cell_val:
                    first_non_empty = cell_val
                    break
            if first_non_empty:
                # Check if the value is a dollar amount.
                if not re.match(r"^\$\s*\d+(?:,\d{3})*(?:\.\d+)?$", first_non_empty):
                    words = first_non_empty.split()
                    if 2 <= len(words) <= 3:
                        # Clean the candidate name.
                        new_name = clean_column_headers([first_non_empty])[0]
                        log_file.write(f"Renaming column '{col}' to '{new_name}' based on first non-empty value '{first_non_empty}'.\n")
                        if new_name in df.columns and new_name != col:
                            # Merge the two columns: fill empty cells in existing new_name from the renamed col.
                            for idx in df.index:
                                existing_val = df.at[idx, new_name]
                                candidate_val = df.at[idx, col]
                                if (pd.isna(existing_val) or existing_val == "") and (not pd.isna(candidate_val) and candidate_val != ""):
                                    df.at[idx, new_name] = candidate_val
                            df.drop(columns=[col], inplace=True)
                        else:
                            df.rename(columns={col: new_name}, inplace=True)
    # Process "Needed For" column: rename or merge it into "description".
    if "Needed For" in df.columns:
        if "description" in df.columns:
            log_file.write("Merging 'Needed For' column into existing 'description' column.\n")
            for idx in df.index:
                desc_val = df.at[idx, "description"]
                needed_for_val = df.at[idx, "Needed For"]
                if (pd.isna(desc_val) or desc_val == "") and (not pd.isna(needed_for_val) and needed_for_val != ""):
                    df.at[idx, "description"] = needed_for_val
            df.drop(columns=["Needed For"], inplace=True)
        else:
            log_file.write("Renaming 'Needed For' column to 'description'.\n")
            df.rename(columns={"Needed For": "description"}, inplace=True)
    return df

def extract_table3(pdf_path, log_file, is_addendum=False):
    """
    Extracts Table 3 data and also tables under Attachment 1/Attachment 2 from the provided PDF.
    Implements:
      1. Refinement of table title when Attachment 1/2 is found by searching the bounding box above the table.
      2. Renaming of duplicate/empty columns (using fix_column_names) and then post-processing
         unnamed columns as described.
    """
    print(f"\nProcessing {pdf_path} for Table 3 / Attachment extraction...", file=log_file)
    extracted_tables = []
    specific_phrase = None
    try:
        with pdfplumber.open(pdf_path) as pdf:
            # Identify pages that contain either Table 3 patterns or Attachment 1/Attachment 2.
            table3_pages = []
            for i, page in enumerate(pdf.pages):
                text = page.extract_text() or ""
                if re.search(r"Table\s*3[-.]([1-3])\b", text, re.IGNORECASE):
                    #re.search(r"Attachment\s*[12]", text, re.IGNORECASE)):
                    table3_pages.append(i)
            if not table3_pages:
                print("No Table 3  found in the PDF.", file=log_file)
                return pd.DataFrame()
            first_page = table3_pages[0]
            last_page = table3_pages[-1]
            scrape_start = first_page
            scrape_end = last_page + 1
            print(f"Candidate pages start on {scrape_start + 1} and end on {scrape_end}", file=log_file)
            # Process each page that might contain table data.
            for page_number in range(scrape_start, min(scrape_end, len(pdf.pages))):
                page = pdf.pages[page_number]
                print(f"\nScraping tables on page {page_number + 1}...", file=log_file)
                # This variable keeps track of the bottom y-coordinate of the previous table on the page.
                previous_table_bottom = None
                tables = page.find_tables(table_settings={
                    "horizontal_strategy": "lines",
                    "vertical_strategy": "lines",
                })
                for table_index, table in enumerate(tables):
                    tab = table.extract()
                    if not tab:
                        print(f"Table {table_index + 1} on page {page_number + 1} is empty. Skipping.", file=log_file)
                        continue
                    table_bbox = table.bbox  # (x0, top, x1, bottom)
                    # Define the title region for the table: above the table bounding box.
                    title_bbox = (0, 0, page.width, table_bbox[1])
                    title_text = page.within_bbox(title_bbox).extract_text() or ""
                    table_title = None
                    if title_text:
                        title_lines = title_text.split('\n')[::-1]
                        for line in title_lines:
                            line = line.strip()
                            match = re.search(r"(Modification\s+of\s+)?Table\s*3[-.]?\d*[:\-\s]*(.*)", line, re.IGNORECASE)
                            if match:
                                table_title = match.group(2).strip()
                                break
                        
                  
                    # Extract the specific phrase using the refined table title.
                    if table_title:
                        specific_phrase = extract_specific_phrase(table_title)
                        print(f"New table detected: '{specific_phrase}' on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        headers = clean_column_headers(tab[0])
                        # Rename header 'type' to 'type of upgrade' if needed.
                        if "type" in headers and "type of upgrade" not in headers:
                            headers = [("type of upgrade" if h == "type" else h) for h in headers]
                        if "need for" in headers:
                            headers = [("description" if h == "need for" else h) for h in headers]  
                    
                        # Apply the duplicate/empty column fixing.
                        headers = fix_column_names(headers)
                        data_rows = tab[1:]
                        try:
                            df_new = pd.DataFrame(data_rows, columns=headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for new table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue

                        if "allocated" in df_new.columns:
                            df_new.drop(columns=["allocated"], inplace=True)
                            print(f"Dropped 'Max of' column in table on page {page_number + 1}, table {table_index}.", file=log_file)

                        if "cost rate x " in df_new.columns:
                            df_new.drop(columns=["cost rate x "], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file)

                        if "cost rate" in df_new.columns:
                            df_new.drop(columns=["cost rate"], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file) 

                        #if "3339615 9" in df_new.columns:
                         #   df_new.drop(columns=["3339615 9"], inplace=True)
                         #   print(f"Dropped '3339615 9' column in table on page {page_number + 1}, table {table_index}.", file=log_file)     
                            
                        if "6 steady state reliability and posttransient voltage stability" in df_new.columns:
                            df_new.drop(columns=["6 steady state reliability and posttransient voltage stability"], inplace=True)
                            print(f"Dropped '6 steady state reliability and posttransient voltage stability' column in table on page {page_number + 1}, table {table_index}.", file=log_file)  



                        if "escalated" in df_new.columns:
                            df_new.drop(columns=["escalated"], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file)    


                        # Also, if the DataFrame has a column named "type" (and not already "type of upgrade"), rename it.
                        if 'type' in df_new.columns and 'type of upgrade' not in df_new.columns:
                            df_new.rename(columns={'type': 'type of upgrade'}, inplace=True)
                        # Special handling for ADNU tables if needed.
                        if re.search(r"Area\s*Delivery\s*Upgrades", specific_phrase, re.IGNORECASE):
                            print("Detected 'Area Delivery Network Upgrade' table (new).", file=log_file)
                            if "adnu" in df_new.columns:
                                if "type of upgrade" not in df_new.columns:
                                    adnu_values = df_new["adnu"].dropna().astype(str).tolist()
                                    grouped_adnu = " ".join(adnu_values)
                                    other_columns = df_new.drop(columns=["adnu"]).iloc[0].to_dict()
                                    df_grouped = pd.DataFrame({
                                        "upgrade": [grouped_adnu],
                                        "type of upgrade": [specific_phrase]
                                    })
                                    for col, value in other_columns.items():
                                        df_grouped[col] = value
                                    print("Grouped all 'adnu' rows into a single 'upgrade' row for new ADNU table.", file=log_file)
                                    df_new = df_grouped
                                else:
                                    if "upgrade" in df_new.columns:
                                        df_new.drop(columns=['adnu'], inplace=True)
                                        print("Dropped 'adnu' column to avoid duplicate 'upgrade'.", file=log_file)
                                    else:
                                        df_new.rename(columns={'adnu': 'upgrade'}, inplace=True)
                                        print("Renamed 'adnu' to 'upgrade' in new ADNU table.", file=log_file)
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new ADNU table.", file=log_file)
                            else:
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                                    print("Replaced None in 'type of upgrade' for new ADNU table.", file=log_file)
                        else:
                            if "type of upgrade" not in df_new.columns:
                                df_new["type of upgrade"] = specific_phrase
                                print("Added 'type of upgrade' to all rows in new non-ADNU table.", file=log_file)
                            else:
                                first_row = df_new.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print("Replacing None in 'type of upgrade' for new non-ADNU table.", file=log_file)
                                    df_new.at[0, "type of upgrade"] = specific_phrase
                        # Fix duplicate and unnamed columns in the new table.
                        df_new.columns = fix_column_names(df_new.columns.tolist())
                        # Now apply the post-processing of column names:
                        df_new = post_process_columns(df_new, log_file)
                        if "cost rate x " in df_new.columns:
                            df_new.drop(columns=["cost rate x "], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file)

                        if "cost rate" in df_new.columns:
                            df_new.drop(columns=["cost rate"], inplace=True)
                            print(f"Dropped 'cost rate x' column in table on page {page_number + 1}, table {table_index}.", file=log_file) 

                        extracted_tables.append(df_new)
                    else:
                        # Continuation table branch.
                        if specific_phrase is None:
                            print(f"No previous table title found for continuation on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue
                        print(f"Continuation table detected on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        data_rows = tab
                        # Use the number of columns from the last extracted table as expected.
                        expected_columns = len(extracted_tables[-1].columns) if extracted_tables else None
                        if expected_columns is None:
                            print(f"No existing table to continue with on page {page_number + 1}, table {table_index + 1}. Skipping.", file=log_file)
                            continue
                        expected_headers = extracted_tables[-1].columns.tolist()
                        header_keywords = ["type of upgrade", "adnu"]
                        first_continuation_row = data_rows[0] if data_rows else []
                        is_header_row = any(
                            re.search(rf"\b{kw}\b", str(cell), re.IGNORECASE) for kw in header_keywords for cell in first_continuation_row
                        )
                        if is_header_row:
                            print(f"Detected header row in continuation table on page {page_number + 1}, table {table_index + 1}.", file=log_file)
                            data_rows = data_rows[1:]
                        # Ensure every row has the same length as expected_headers.
                        adjust_rows_length(data_rows, expected_headers)
                        try:
                            df_continuation = pd.DataFrame(data_rows, columns=expected_headers)
                        except ValueError as ve:
                            print(f"Error creating DataFrame for continuation table on page {page_number + 1}, table {table_index + 1}: {ve}", file=log_file)
                            continue
                        # Rename column 'type' if needed.
                        if 'type' in df_continuation.columns and 'type of upgrade' not in df_continuation.columns:
                            df_continuation.rename(columns={'type': 'type of upgrade'}, inplace=True)
                        if "need for" in df_continuation.columns:
                            df_continuation.rename(columns={"need for": "description"}, inplace=True)
                        if re.search(r"Area\s*Delivery\s*Upgrades", specific_phrase, re.IGNORECASE):
                            if "type of upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing None in 'type of upgrade' for continuation ADNU table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "type of upgrade"] = specific_phrase
                            else:
                                df_continuation["type of upgrade"] = specific_phrase
                                print(f"'type of upgrade' column added with value '{specific_phrase}' for continuation ADNU table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        else:
                            if "type of upgrade" in df_continuation.columns:
                                first_row = df_continuation.iloc[0]
                                if pd.isna(first_row["type of upgrade"]) or first_row["type of upgrade"] == "":
                                    print(f"Replacing None in 'type of upgrade' for continuation table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                                    df_continuation.at[0, "type of upgrade"] = specific_phrase
                            else:
                                df_continuation["type of upgrade"] = specific_phrase
                                print(f"'type of upgrade' column added with value '{specific_phrase}' for continuation table on page {page_number + 1}, table {table_index + 1}", file=log_file)
                        # Fix duplicate and unnamed columns in the continuation table.
                        df_continuation.columns = fix_column_names(df_continuation.columns.tolist())
                        # Post-process the columns in the continuation table.
                        df_continuation = post_process_columns(df_continuation, log_file)
                        # Concatenate the continuation table with the previous extracted table.
                        extracted_tables[-1] = pd.concat([extracted_tables[-1], df_continuation], ignore_index=True, sort=False)
                    # Update the previous_table_bottom for the page using the current table's bbox.
                    previous_table_bottom = table_bbox[3]
    except Exception as e:
        print(f"Error processing Table 3 in {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()
    if extracted_tables:
        all_columns = set()
        for df in extracted_tables:
            all_columns.update(df.columns.tolist())
        standardized_tables = []
        for df in extracted_tables:
            standardized_df = df.reindex(columns=all_columns)
            standardized_tables.append(standardized_df)
        print("\nConcatenating all extracted Table 3/Attachment data...", file=log_file)
        try:
            table3_data = pd.concat(standardized_tables, ignore_index=True, sort=False)
            print(f"Successfully concatenated {len(standardized_tables)} tables.", file=log_file)
        except Exception as e:
            print(f"Error concatenating tables: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            table3_data = pd.DataFrame()
    else:
        print("No Table 3/Attachment data extracted.", file=log_file)
        table3_data = pd.DataFrame()
    return table3_data


def extract_table3_and_replace_none(pdf_path, project_id, log_file, is_addendum=False):
    """Extracts Table 3 data and merges with base data."""
    base_data = extract_base_data(pdf_path, project_id, log_file)
    table3_data = extract_table3(pdf_path, log_file, is_addendum)
    if table3_data.empty:
        return base_data
    else:
        overlapping_columns = base_data.columns.intersection(table3_data.columns).difference(['point_of_interconnection'])
        table3_data = table3_data.drop(columns=overlapping_columns, errors='ignore')
        base_data_repeated = pd.concat([base_data] * len(table3_data), ignore_index=True)
        try:

                        # Concatenate base data with Table 8 data along columns
            merged_df = pd.concat([base_data_repeated, table3_data], axis=1, sort=False)
           # if "q_id" in merged_df.columns and "type of upgrade" in merged_df.columns and "upgrade" in merged_df.columns:
           #     merged_df["is_duplicate"] = merged_df.duplicated(subset=["q_id", "type of upgrade", "upgrade"], keep="first")
            #    merged_df = merged_df[merged_df["is_duplicate"] == False].drop(columns=["is_duplicate"])
            #    print(f"Removed duplicate rows based on 'q_id', 'type of upgrade', and 'upgrade'.", file=log_file)


            if "q_id" in merged_df.columns and "type of upgrade" in merged_df.columns and "upgrade" in merged_df.columns:
                # Identify rows where 'type of upgrade' and 'upgrade' are not empty
                non_empty_rows = merged_df[
                    merged_df["type of upgrade"].notna() & merged_df["upgrade"].notna() &
                    (merged_df["type of upgrade"].str.strip() != "") & (merged_df["upgrade"].str.strip() != "")
                ]

                # Group by q_id, type of upgrade, and upgrade, keeping the first occurrence
                grouped_df = non_empty_rows.groupby(["q_id", "type of upgrade", "upgrade"], as_index=False).first()

                # Get the original order of the rows in merged_df before filtering
                merged_df["original_index"] = merged_df.index

                # Combine unique grouped rows with originally empty rows
                final_df = pd.concat([
                    grouped_df,
                    merged_df[merged_df["type of upgrade"].isna() | (merged_df["type of upgrade"].str.strip() == "") |
                            merged_df["upgrade"].isna() | (merged_df["upgrade"].str.strip() == "")]
                ], ignore_index=True, sort=False)

                # Restore the original order of the rows based on the saved index
                final_df.sort_values(by="original_index", inplace=True)
                final_df.drop(columns=["original_index"], inplace=True)
                merged_df = final_df

                print(f"Removed duplicate rows based on 'q_id', 'type of upgrade', and 'upgrade', excluding empty rows while preserving order.", file=log_file)

            merged_df = pd.concat([base_data_repeated, table3_data], axis=1, sort=False)
            if 'point_of_interconnection' not in merged_df.columns:
                merged_df['point_of_interconnection'] = base_data['point_of_interconnection'].iloc[0]
                print(f"Added 'point_of_interconnection' to merged data for {pdf_path}.", file=log_file)
            print(f"Merged base data with Table 3 data for {pdf_path}.", file=log_file)
            return merged_df
        except Exception as e:
            print(f"Error merging base data with Table 3 data for {pdf_path}: {e}", file=log_file)
            print(traceback.format_exc(), file=log_file)
            return base_data

def check_has_table3(pdf_path):
    """Checks if the PDF contains Table 3 or Attachment 1/Attachment 2."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for page in pdf.pages:
                text = page.extract_text() or ""
                if re.search(r"(Modification\s+of\s+)?Table\s*3[-.]?\d*", text, re.IGNORECASE): 
                    #re.search(r"Attachment\s*[12]", text, re.IGNORECASE))
                    return True
    except Exception as e:
        return False
    return False

def is_addendum(pdf_path):
    """Checks if the PDF is an addendum by searching 'Addendum' on the first page."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            if len(pdf.pages) == 0:
                return False
            first_page = pdf.pages[0]
            text = first_page.extract_text() or ""
            return "Addendum" in text
    except Exception as e:
        return False

def extract_base_data(pdf_path, project_id, log_file):
    """Extract base data from the PDF and return as a DataFrame."""
    print("Extracting base data from PDF...", file=log_file)
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text
        text = clean_string_cell(text)
        queue_id = re.search(r"q[\s_-]*(\d+)", text, re.IGNORECASE)
        queue_id = queue_id.group(1) if queue_id else str(project_id)
        print(f"Extracted Queue ID: {queue_id}", file=log_file)
        cluster_number = re.search(r"queue[\s_-]*cluster[\s_-]*(\d+)", text, re.IGNORECASE)
        cluster_number = cluster_number.group(1) if cluster_number else None
        print(f"Extracted Cluster Number: {cluster_number}", file=log_file)
        deliverability_status = re.search(r"(\w+)\s*capacity deliverability status", text, re.IGNORECASE)
        deliverability_status = deliverability_status.group(1) if deliverability_status else None
        print(f"Extracted Deliverability Status: {deliverability_status}", file=log_file)
        capacity = re.search(r"total rated output of (\d+)\s*mw", text, re.IGNORECASE)
        if capacity:
            capacity = int(capacity.group(1))
        else:
            capacity2 = re.search(r"(\d+)\s*mw", text)
            capacity = int(capacity2.group(1)) if capacity2 else None
        print(f"Extracted Capacity: {capacity}", file=log_file)
        point_of_interconnection = extract_table2(pdf_path, log_file)
        latitude, longitude = search_gps_coordinates(text, log_file)
        base_data = {
            "q_id": [queue_id],
            "cluster": [cluster_number],
            "req_deliverability": [deliverability_status],
            "latitude": [latitude],
            "longitude": [longitude],
            "capacity": [capacity],
            "point_of_interconnection": [point_of_interconnection]
        }
        print("Base data extracted:", file=log_file)
        print(base_data, file=log_file)
        return pd.DataFrame(base_data)
    except Exception as e:
        print(f"Error extracting base data from {pdf_path}: {e}", file=log_file)
        print(traceback.format_exc(), file=log_file)
        return pd.DataFrame()

def save_to_csv(df, output_csv_path, data_type):
    """Cleans the DataFrame and saves it to a CSV file."""
    if df.empty:
        print(f"No data to save for {data_type}.")
        return
    df = df.map(clean_string_cell)
    df = df[~df.apply(lambda row: contains_phrase(row, "Type of Upgrade"), axis=1)]
    df = reorder_columns(df)
    print(f"\nColumns reordered for {data_type} as per specification.")
    if 'q_id' in df.columns:
        df['q_id'] = pd.to_numeric(df['q_id'], errors='coerce')
    try:
        df.to_csv(output_csv_path, index=False)
        print(f"\nData successfully saved to {output_csv_path}")
    except Exception as e:
        print(f"Error saving {data_type} data to CSV: {e}")
        print(traceback.format_exc())

def process_pdfs_in_folder():
    """Processes all PDFs in the specified project range and directory."""
    global core_originals, core_addendums, total_pdfs_accessed, total_pdfs_scraped, total_pdfs_skipped
    os.makedirs(os.path.dirname(LOG_FILE_PATH), exist_ok=True)
    with open(LOG_FILE_PATH, 'w') as log_file:
        for project_id in PROJECT_RANGE:
            project_path = os.path.join(BASE_DIRECTORY, str(project_id), "02_phase_1_study")
            if not os.path.exists(project_path):
                missing_projects.add(project_id)
                print(f"Project path does not exist: {project_path}", file=log_file)
                continue
            project_scraped = False
            base_data_extracted = False
            base_data = pd.DataFrame()
            for pdf_name in os.listdir(project_path):
                if pdf_name.endswith(".pdf"):
                    pdf_path = os.path.join(project_path, pdf_name)
                    total_pdfs_accessed += 1
                    is_add = is_addendum(pdf_path)
                    if is_add:
                        addendum_pdfs.append(pdf_name)
                        print(f"Accessing Addendum PDF: {pdf_name} from Project {project_id}", file=log_file)
                    else:
                        original_pdfs.append(pdf_name)
                        print(f"Accessing Original PDF: {pdf_name} from Project {project_id}", file=log_file)
                    try:
                        has_table3 = check_has_table3(pdf_path)
                        if not has_table3:
                            skipped_pdfs.append(pdf_name)
                            print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3 or Attachment data)", file=log_file)
                            print(f"Skipped PDF: {pdf_name} from Project {project_id} (No Table 3 or Attachment data)")
                            total_pdfs_skipped += 1
                            continue
                        if not is_add and not base_data_extracted:
                            base_data = extract_base_data(pdf_path, project_id, log_file)
                            base_data_extracted = True
                            print(f"Extracted base data from original PDF: {pdf_name}", file=log_file)
                        if is_add and base_data_extracted:
                            table3_data = extract_table3(pdf_path, log_file, is_addendum=is_add)
                            if not table3_data.empty:
                                merged_df = pd.concat([base_data] * len(table3_data), ignore_index=True)
                                merged_df = pd.concat([merged_df, table3_data], axis=1, sort=False)
                                core_addendums = pd.concat([core_addendums, merged_df], ignore_index=True)
                                scraped_pdfs.append(pdf_name)
                                scraped_projects.add(project_id)
                                project_scraped = True
                                total_pdfs_scraped += 1
                                print(f"Scraped Addendum PDF: {pdf_name} from Project {project_id}")
                            else:
                                skipped_pdfs.append(pdf_name)
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                                print(f"Skipped Addendum PDF: {pdf_name} from Project {project_id} (Empty Data)")
                                total_pdfs_skipped += 1
                        else:
                            df = extract_table3_and_replace_none(pdf_path, project_id, log_file, is_addendum=is_add)
                            if not df.empty:
                                if is_add:
                                    core_addendums = pd.concat([core_addendums, df], ignore_index=True)
                                else:
                                    core_originals = pd.concat([core_originals, df], ignore_index=True)
                                scraped_pdfs.append(pdf_name)
                                scraped_projects.add(project_id)
                                project_scraped = True
                                total_pdfs_scraped += 1
                                print(f"Scraped PDF: {pdf_name} from Project {project_id}")
                            else:
                                skipped_pdfs.append(pdf_name)
                                print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)", file=log_file)
                                print(f"Skipped PDF: {pdf_name} from Project {project_id} (Empty Data)")
                                total_pdfs_skipped += 1
                    except Exception as e:
                        skipped_pdfs.append(pdf_name)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}", file=log_file)
                        print(traceback.format_exc(), file=log_file)
                        print(f"Skipped PDF: {pdf_name} from Project {project_id} due to an error: {e}")
                        total_pdfs_skipped += 1
            if not project_scraped and os.path.exists(project_path):
                skipped_projects.add(project_id)
    save_to_csv(core_originals, OUTPUT_CSV_PATH_ORIGINAL, "originals")
    save_to_csv(core_addendums, OUTPUT_CSV_PATH_ADDENDUM, "addendums")
    total_projects_processed = len(scraped_projects) + len(skipped_projects)
    print("\n=== Scraping Summary ===")
    print(f"Total Projects Processed: {total_projects_processed}")
    print(f"Total Projects Scraped: {len(scraped_projects)}")
    print(f"Total Projects Skipped: {len(skipped_projects)}")
    print(f"Total Projects Missing: {len(missing_projects)}")
    print(f"Total PDFs Accessed: {total_pdfs_accessed}")
    print(f"Total PDFs Scraped: {total_pdfs_scraped}")
    print(f"Total PDFs Skipped: {total_pdfs_skipped}")
    print("\nList of Scraped Projects:")
    print(sorted(scraped_projects))
    print("\nList of Skipped Projects:")
    print(sorted(skipped_projects))
    print("\nList of Missing Projects:")
    print(sorted(missing_projects))
    print("\nList of Scraped PDFs:")
    print(scraped_pdfs)
    print("\nList of Skipped PDFs:")
    print(skipped_pdfs)
    print("\nList of Addendum PDFs:")
    print(addendum_pdfs)
    print("\nList of Original PDFs:")
    print(original_pdfs)
    print("\nNumber of Original PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in original_pdfs]))
    print("Number of Addendum PDFs Scraped:", len([pdf for pdf in scraped_pdfs if pdf in addendum_pdfs]))

def main():
    """Main function to execute the PDF scraping process."""
    process_pdfs_in_folder()

if __name__ == "__main__":
    main()


Skipped PDF: 11AS763732-C5PIEastQ871AppendixA.pdf from Project 871 (No Table 3 or Attachment data)
Skipped PDF: 11AS763774-C5PIMetroQ893AppA.pdf from Project 873 (No Table 3 or Attachment data)
Scraped PDF: 12AS779216-Appendix_AQ877_California_Flats_Solar_C5_Ph_I_Study_Report.pdf from Project 877
Scraped PDF: 12AS779571-Appendix_AQ885_SKIC_Solar_C5_Ph_I_Final_Study_Report_CMB_31JAN2013.pdf from Project 885
Scraped PDF: 12AS780495-QC5PIEOPQ887AppendixA.pdf from Project 887
Skipped PDF: 12AS780604-QC5PINOLQ888Appendix_A.pdf from Project 888 (No Table 3 or Attachment data)
Skipped PDF: 12AS778778-QC5PINorthernQ890Backus_Solar_EnergyAppendix_A.pdf from Project 890 (No Table 3 or Attachment data)
Scraped PDF: 12AS782071-QC5PIEOPQ891AppendixA.pdf from Project 891
Scraped PDF: 12AS780639-QC5PIEOPQ892AppendixA.pdf from Project 892
Skipped PDF: 12AS780344-C5PIMetroQ893AppA.pdf from Project 893 (No Table 3 or Attachment data)
Scraped PDF: 12AS779610-Appendix_AQ894_Prospect_Energy.pdf from Projec

# Code to create clean and itemized datasets

In [111]:
import pandas as pd
import re
import unicodedata

# Load the CSV file
df = pd.read_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/03_raw/rawdata_cluster5_style_D_originals.csv', dtype={'estimated_time_to_construct': str})

df['q_id'] = df['q_id'].astype('Int64')
df['cluster'] = df['cluster'].astype('Int64')


######################################################################################################################################
########################################
# STEP 0: CREATE DESCRIPTION COLUMN FROM COST ALLOCATION FACTOR


def move_non_numeric_text(value):
    """Move non-numeric, non-percentage text from cost allocation factor to description.
       If a value is moved, return None for cost allocation factor."""
    if isinstance(value, str):  # Ensure it's a string before processing
        if re.fullmatch(r"[\d,.]+%?", value):  # Check if it's numeric or a percentage
            return value  # Keep numeric or percentage values
        return None  # Clear the value if it's text (moved to description)
    return value  # Return as is for non-string values


def extract_non_numeric_text(value):
    """Extract non-numeric, non-percentage text from the cost allocation factor column."""
    if isinstance(value, str):  # Ensure it's a string before processing
        if re.fullmatch(r"[\d,.]+%?", value):  # Check if it's numeric or a percentage
            return None
        return value.strip()  # Return text entries as is
    return None  # Return None for non-string values



def clean_total_entries(value):
    """If the value starts with 'Total', remove numbers, commas, and percentage signs, keeping only 'Total'."""
    if isinstance(value, str) and value.startswith("Total"):
        return "Total"  # Keep only "Total"
    return value  # Leave other values unchanged









# Create the 'description' column from 'cost allocation factor'
#if 'cost allocation factor' in df.columns:
#    df['description'] = df['cost allocation factor'].apply(extract_non_numeric_text)
#    df['cost_allocation_factor'] = df['cost allocation factor'].apply(move_non_numeric_text)  # Clear moved values

######################################################################################################################################
########################################
#STEP 1 MERGE COLUMNS

def merge_columns(df):
    merge_columns_dict = {

        "upgrade": [
            "upgrade",
            "unnamed_1",
            ],

        "capacity": [
            "capacity",
            "MW",
            "unnamed_9",
            "mw",
        ],   



        "estimated_cost_x_1000": [

            "estimated cost x 1000",
            "estimated cost x 1000 constant",
            "allocated_cost",
            "assigned cost",
            "allocated cost",
            "sum of allocated constant cost",
            "estimated cost x 1000 constant dollar",
        ],    


        "escalated_cost_x_1000": [
            "escalated costs x 1000",
            "escalated cost x 1000 constant dollar",
            "estimated cost x 1000 escalated",
            "allocated cost escalated",
            "estimated cost x 1000 escalated without itcca",
            "escalated cost x 1000",
            "sum of allocated escalated cost",
            "assigned cost escalated",
            "unnamed_17",
            "unnamed_18",
            "3339615 9"
             

        ],

        "estimated_time_to_construct": [
            "estimated time to construct",
            "estimated time  to construct",
            


        ],

        "total_estimated_cost_x_1000": [
            "total nu cost",
            "total cost constant"
        ],
        "total_estimated_cost_x_1000_escalated": [
            "total estimated cost x 1000 escalalted",
            "total estimated cost x 1000 escalated"
        ],
       
         

        "description": ["description", "unnamed_7"],

        
        "cost_allocation_factor": [
            "cost allocation factor",
            "cost allocatio n factor",
            "cost allocati on factor",
            "project allocation",

        ],
       
    }

    # Identify unnamed columns
    unnamed_columns = [col for col in df.columns if pd.isna(col) or col.strip() == "" or col.startswith("Unnamed")]
    if unnamed_columns:
        merge_columns_dict["description"].extend(unnamed_columns)

    for new_col, old_cols in merge_columns_dict.items():
        existing_cols = [col for col in old_cols if col in df.columns]
        if existing_cols:
            df[new_col] = df[existing_cols].bfill(axis=1).iloc[:, 0]
            cols_to_drop = [col for col in existing_cols if col != new_col]
            if cols_to_drop:
                df.drop(columns=cols_to_drop, inplace=True)

    return df

df = merge_columns(df)


######################################################################################################################################
########################################
# STEP 2: REMOVE DOLLAR SIGNED VALUES FROM 'estimated_time_to_construct'
######## Other clean up

def remove_dollar_values(value):
    """Remove dollar amounts (e.g., $3625.89, $3300) from 'estimated_time_to_construct'."""
    if isinstance(value, str) and re.search(r"^\$\d+(\.\d{1,2})?$", value.strip()):
        return None  # Replace with None if it's a dollar-signed number
    return value.strip() if isinstance(value, str) else value

if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(remove_dollar_values)


## Remove ranodm number in Total row:    
# Apply cleaning function to "upgrade" column after merging
#if 'upgrade' in df.columns:
 #   df['upgrade'] = df['upgrade'].apply(clean_total_entries)


 
 

    
######################################################################################################################################
########################################
# STEP 3: DROP UNNEEDED COLUMNS

#df.drop(['unnamed_3', 'unnamed_15', 'unnamed_18', 'unnamed_16', 'estimated cost x 1000 escalated with itcca'], axis=1, inplace=True, errors='ignore')

df.drop(['unnamed_2','unnamed_3', 'unnamed_4','unnamed_5','unnamed_6','unnamed_10', 'unnamed_11','unnamed_12','unnamed_13','unnamed_14', 'unnamed_15','unnamed_16', 
           'unnamed_19', 'unnamed_20', 'cost rate x 1000mw constant dollar', 'needed for',
         'cost rate x'], axis=1, inplace=True)



######################################################################################################################################
########################################
#STEP 4: NAMING CONVENTION
def convert_to_snake_case(column_name):
    column_name = column_name.strip().lower()
    column_name = re.sub(r'[\s\-]+', '_', column_name)
    column_name = re.sub(r'[^\w]', '', column_name)
    return column_name

def clean_string_cell(value):
    if isinstance(value, str):
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
        value = value.replace('\n', ' ').strip()
    return value

df = df.map(clean_string_cell)
df.columns = [convert_to_snake_case(col) for col in df.columns]



# Convert estimated_time_to_construct to integer (remove decimals) and keep NaNs as empty
df['estimated_time_to_construct'] = pd.to_numeric(df['estimated_time_to_construct'], errors='coerce').apply(lambda x: int(x) if pd.notna(x) else None)


 






######################################################################################################################################
########################################
#STEP 5: REMOVING TOTAL ROW, AS THE PDFS GIVE TOTAL NETWORK COST RATHER THAN BY RNU, LDNU AS WE HAD BEFORE
# Remove rows where upgrade is "Total" (case-insensitive)

df['tot'] = df.apply(
    lambda row: 'yes' if (
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor'])) 
        ) else 'no',
    axis=1
) 

# Now extract ONLY "Total" rows with a foolproof match
total_rows_df = df[df['tot'] == 'yes']

total_rows_df = total_rows_df.groupby(['q_id', 'type_of_upgrade', 'upgrade'], as_index=False).first()
total_rows_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_total_network.csv', index=False) 
df = df[df['cost_allocation_factor'].str.strip().str.lower() != 'total']
 
df.drop('tot', axis=1, inplace= True)




######################################################################################################################################
########################################
# STEP 6: Move upgrade phrases like IRNU from upgrade column to a new column upgrade_classificatio and also replace type_of_upgrade with LDNU, CANU



# Define the list of phrases for upgrade classification
upgrade_phrases = ["IRNU", "GRNU", "CANU-D", "IRNU-A", "LDNU", "CANU-GR", "PNU", "CANU"]

 



#df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 14/03_raw/cluster_14_style_Q.csv', index=False)  


if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )


mappings = {
    "PTO’s Interconnection Facilities (Note 2)": "PTO_IF",
    "PTO’s Interconnectio n Facilities (Note 2)": "PTO_IF",
    "PTOs Interconnection Facilities": "PTO_IF",
    "PTOs Interconnectio n Facilities": "PTO_IF",
 "PTO": 'PTO_IF',
 "PNU": "OPNU",
 "Delivery Network Upgrades": "LDNU",
 "Reliability Network Upgrades": "RNU",
    "Local Delivery Network Upgrades": "LDNU",
    "Area Deliverability Upgrades": "ADNU",
    "Escalated Cost and Time to Construct for Interconnection Facilities, Reliability Network Upgrades, and Delivery Network Upgrades": "LDNU",
    "Distribution": "ADNU",
'Total PTO_IF': 'PTO_IF',
 'Total RNU': 'RNU',
 'Total LDNU': 'LDNU',
 'Total OPNU' : 'OPNU',
 'Total CANU': 'CANU',
 'Total LOPNU': 'LOPNU',
 'Total ADNU': 'ADNU',
}


def reorder_columns(df):
    """
    Reorders the columns of the DataFrame based on the specified order.

    Args:
        df (pd.DataFrame): The DataFrame to reorder.

    Returns:
        pd.DataFrame: The reordered DataFrame.
    """
    desired_order = [
        "q_id",
        "cluster",
        "req_deliverability",
        "latitude",
        "longitude",
        "capacity",
        "point_of_interconnection",
        "type_of_upgrade",
        "upgrade",
        "description",
        "cost_allocation_factor",
        "estimated_cost_x_1000",
        "escalated_cost_x_1000",
        "total_estimated_cost_x_1000",
        "total_estimated_cost_x_1000_escalated",
        "estimated_time_to_construct",
    ]

    # Start with desired columns that exist in the DataFrame
    existing_desired = [col for col in desired_order if col in df.columns]

    # Then add the remaining columns
    remaining = [col for col in df.columns if col not in existing_desired]

    # Combine the two lists
    new_order = existing_desired + remaining

    # Reorder the DataFrame
    df = df[new_order]

    return df     


if 'type_of_upgrade' in df.columns:
  
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )


if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].ffill()    

df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/03_raw/cluster_5_style_D.csv', index=False)

######################################################################################################################################
########################################
#STEP 7: Stable sort type of upgrade

def stable_sort_by_type_of_upgrade(df):
    """Performs a stable sort within each q_id to order type_of_upgrade while preserving row order in other columns."""
    
    # Define the custom sorting order for type_of_upgrade
    type_order = {"PTO_IF": 1, "RNU": 2, "LDNU": 3, "PNU": 4, "ADNU": 5}

    # Assign a numerical sorting key; use a high number if type_of_upgrade is missing
    df['sort_key'] = df['type_of_upgrade'].map(lambda x: type_order.get(x, 99))

    # Perform a stable sort by q_id first, then by type_of_upgrade using the custom order
    df = df.sort_values(by=['q_id', 'sort_key'], kind='stable').drop(columns=['sort_key'])

    return df

# Apply stable sorting
  


df = df.groupby(['q_id', 'type_of_upgrade', 'upgrade'], as_index=False).first()


df= reorder_columns(df)

#df = stable_sort_by_type_of_upgrade(df)  
#df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 14/03_raw/cluster_14_style_Q.csv', index=False)
######################################################################################################################################
########################################
# STEP 8: Remove $ signs and convert to numeric
 

def clean_currency(value):
    """
    Cleans a string by explicitly removing $, *, (Note 2), and similar patterns,
    then converts it to a numeric value.
    """
    if isinstance(value, str):
        # Explicitly remove $, *, and any "(Note ...)"
        value = value.replace('$', '').replace('*', '')
        value = re.sub(r'\(Note \d+\)', '', value)  # Remove patterns like "(Note 2)"
        value = value.replace(',', '').strip()  # Remove commas and extra spaces
    try:
        return pd.to_numeric(value)
    except ValueError:
        return pd.NA  # Return NaN for invalid entries
    




# Clean the specific columns
for col in ['estimated_cost_x_1000', 'escalated_cost_x_1000',]:
    if col in df.columns:
        df[col] = df[col].apply(clean_currency)






######################################################################################################################################
########################################
# STEP 9: Create Total rows


df['item'] = df.apply(
    lambda row: 'no' if (
        (pd.notna(row.get('type_of_upgrade')) and 'Total' in str(row['type_of_upgrade'])) or
        (pd.notna(row.get('cost_allocation_factor')) and 'Total' in str(row['cost_allocation_factor']))
    ) else 'yes',
    axis=1
)


  

# Create Total rows
new_rows = []
columns_to_sum = ['estimated_cost_x_1000', 'escalated_cost_x_1000',]
columns_to_populate = ['cluster', 'req_deliverability', 'latitude', 'longitude', 'capacity', 'point_of_interconnection']

for q_id, group in df.groupby('q_id', as_index=False):
    print(f"\nProcessing q_id: {q_id}")  # Debug print
    unique_upgrades = group['type_of_upgrade'].dropna().unique()
    for upgrade in unique_upgrades:
        if pd.isna(upgrade):
            continue
        
        rows = group[group['type_of_upgrade'] == upgrade]

        # Debug: Print current group
        print(f"\nChecking Upgrade: {upgrade}, Total Rows Present?:", 
              ( (group['item'] == 'no')).any())

        # Check if a Total row already exists for this (q_id, upgrade)
        total_exists = ((group['item'] == 'no')).any()
        
        if total_exists:
            print(f"Skipping Total row for {upgrade} (already exists).")
            continue
        
        total_row = {col: '' for col in df.columns}  # Initialize all columns as empty strings
        total_row['q_id'] = q_id
        total_row['type_of_upgrade'] = f"Total {upgrade}"
        total_row['item'] = 'no'

        # Populate specified columns from the existing row
        first_row = rows.iloc[0]
        for col in columns_to_populate:
            if col in df.columns:
                total_row[col] = first_row[col]

        # Sum the numeric columns
        for col in columns_to_sum:
            if col in rows.columns:
                total_row[col] = rows[col].sum()
            else:
                total_row[col] = 0  # Default to 0 if column is missing

        print(f"Creating Total row for {upgrade}")  # Debug print
        new_rows.append(total_row)

# Convert list to DataFrame and append
if new_rows:
    total_rows_df = pd.DataFrame(new_rows)
    print("\nNew Total Rows Created:\n", total_rows_df)  # Debug print
    df = pd.concat([df, total_rows_df], ignore_index=True)

df.reset_index(drop=True, inplace=True)


if 'type_of_upgrade' in df.columns:
  
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: mappings.get(x, x) if isinstance(x, str) else x
    )


df = stable_sort_by_type_of_upgrade(df)

 


#: Move 'item' column next to 'type_of_upgrade'
if 'item' in df.columns and 'type_of_upgrade' in df.columns:
    cols = df.columns.tolist()
    item_index = cols.index('item')
    type_index = cols.index('type_of_upgrade')
    if item_index < type_index:
        cols.insert(type_index + 1, cols.pop(item_index))
    else:
        cols.insert(type_index + 1, cols.pop(item_index))
    df = df[cols]




#  Remove "Total" values from cost_allocation_factor if they appear in type_of_upgrade
if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if (
            pd.notna(row['type_of_upgrade']) and 'Total' in str(row['type_of_upgrade'])
        ) else row.get('cost_allocation_factor'),
        axis=1
    )
    

if 'cost_allocation_factor' in df.columns and 'type_of_upgrade' in df.columns:
    df['cost_allocation_factor'] = df.apply(
        lambda row: None if 'Total' in str(row.get('cost_allocation_factor', '')) else row.get('cost_allocation_factor'),
        axis=1
    )



def clean_estimated_time(value):
    if isinstance(value, str):
        value = re.sub(r'(\d+(?:-\w+)*)\s+\w+.*$', r'\1', value, flags=re.IGNORECASE).strip()
    return value

if 'estimated_time_to_construct' in df.columns:
    df['estimated_time_to_construct'] = df['estimated_time_to_construct'].apply(clean_estimated_time)

if 'type_of_upgrade' in df.columns:
    df['type_of_upgrade'] = df['type_of_upgrade'].apply(
        lambda x: re.sub(r'\s*\(note \d+\)', '', x, flags=re.IGNORECASE).strip() if isinstance(x, str) else x
    )

if 'type_of_upgrade' in df.columns:
    previous_type_of_upgrade = None
    for i in range(len(df)):
        if df.at[i, 'type_of_upgrade'] == 'Total':
            if previous_type_of_upgrade is not None:
                df.at[i, 'type_of_upgrade'] = previous_type_of_upgrade
        else:
            previous_type_of_upgrade = df.at[i, 'type_of_upgrade']

numeric_columns = [
    'cost_allocation_factor',
    'estimated_cost_x_1000',
    'estimated_time_to_construct',
    'total_estimated_cost_x_1000_escalated',
    'adnu_cost_rate_x_1000',
    'escalated_cost_x_1000',
    'estimated_cost_x_1000_escalated_without_itcca',
    'adnu_cost_rate_x_1000_escalated'
]
non_numeric_columns = ['type_of_upgrade', 'upgrade', 'description']

for col in non_numeric_columns:
    if col in df.columns:
        df[col] = df[col].apply(lambda x: 'None' if pd.isna(x) else x)

for col in numeric_columns:
    if col in df.columns:
        df[col] = df[col].replace('-', pd.NA)
        df[col] = df[col].fillna(0)

if 'original_order' in df.columns and 'q_id' in df.columns:
    df['original_order'] = df.index
    df = df.sort_values(by=['q_id', 'original_order'], ascending=[True, True])
    df = df.drop(columns=['original_order'])


   


#df= reorder_columns(df)


# Save itemized and totals separately
if 'item' in df.columns:
    itemized_df = df[df['item'] == 'yes']
    itemized_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_itemized.csv', index=False)

    totals_columns = ['upgrade', 'description', 'cost_allocation_factor', 'estimated_time_to_construct']
    totals_df = df[df['item'] == 'no'].drop(columns=totals_columns, errors='ignore')
    totals_df.to_csv('/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_total.csv', index=False)

print(f"Itemized rows saved to 'costs_phase_1_cluster_5_style_D_itemized.csv'.")
print(f"Filtered Total rows saved to 'costs_phase_1_cluster_5_style_D_total.csv'.")



if 'type_of_upgrade' in df.columns:
    print(df['type_of_upgrade'].unique())

if 'q_id' in df.columns:
    print(df['q_id'].unique())

if 'cluster' in df.columns:
    print(df['cluster'].unique())


#df.to_csv('Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 14/03_raw/rawdata_cluster14_style_Q.csv')
#



Processing q_id: 877

Checking Upgrade: ADNU, Total Rows Present?: False
Creating Total row for ADNU

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Checking Upgrade: RNU, Total Rows Present?: False
Creating Total row for RNU

Processing q_id: 885

Checking Upgrade: ADNU, Total Rows Present?: False
Creating Total row for ADNU

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Processing q_id: 894

Checking Upgrade: ADNU, Total Rows Present?: False
Creating Total row for ADNU

Checking Upgrade: LDNU, Total Rows Present?: False
Creating Total row for LDNU

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Checking Upgrade: RNU, Total Rows Present?: False
Creating Total row for RNU

Processing q_id: 898

Checking Upgrade: ADNU, Total Rows Present?: False
Creating Total row for ADNU

Checking Upgrade: PTO_IF, Total Rows Present?: False
Creating Total row for PTO_IF

Checking Upgrade: R

/var/folders/l5/3g5pj1nj2j108yb2yjngfmcmcmdwpt/T/ipykernel_65073/3529828934.py:567: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(0)


In [112]:
import pandas as pd
import os
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# ---------------------- Configuration ---------------------- #

# Paths to the CSV files
ITEMIZED_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_itemized.csv'
TOTALS_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_total.csv'
TOTAL_NETWORK_CSV_PATH = "/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_total_network.csv"


# Columns in totals_df that hold the reported total costs
TOTALS_ESTIMATED_COLUMN = 'estimated_cost_x_1000'
TOTALS_ESCALATED_COLUMN = 'escalated_cost_x_1000'

# Upgrade types to check
REQUIRED_UPGRADES = ['PTO_IF', 'RNU', 'LDNU', 'ADNU', 'CANU', "OPNU"]

# Output paths
MISMATCHES_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/mismatches.csv'
MATCHED_QIDS_CSV_PATH = '/Users/vk365/Dropbox/Interconnections_data/data/ic_studies/raw/04_intermediate_scraped_data/phase_1_cost_data/Cluster 5/matched_qids.csv'

# ---------------------- Load Data ---------------------- #

def load_csv(path, dataset_name):
    """
    Loads a CSV file into a pandas DataFrame.
    """
    try:
        df = pd.read_csv(path)
        print(f"Loaded {dataset_name} from {path}")
        return df
    except FileNotFoundError:
        print(f"Error: File not found at {path}")
        exit(1)
    except Exception as e:
        print(f"Error loading {dataset_name}: {e}")
        exit(1)

# Load datasets
itemized_df = load_csv(ITEMIZED_CSV_PATH, "itemized data")
totals_df = load_csv(TOTALS_CSV_PATH, "totals data")
total_network_df = load_csv(TOTAL_NETWORK_CSV_PATH, "total network data")

# ---------------------- Data Cleaning ---------------------- #

def clean_text(df, column):
    """
    Cleans text data by stripping leading/trailing spaces and converting to uppercase.
    """
    if column in df.columns:
        df[column] = df[column].astype(str).str.strip().str.upper()
    else:
        print(f"Warning: '{column}' column is missing in the dataset. Filling with 'UNKNOWN'.")
        df[column] = 'UNKNOWN'
    return df

# Clean 'type_of_upgrade' and 'point_of_interconnection' in both datasets
itemized_df = clean_text(itemized_df, 'type_of_upgrade')
itemized_df = clean_text(itemized_df, 'point_of_interconnection')

totals_df = clean_text(totals_df, 'type_of_upgrade')
totals_df = clean_text(totals_df, 'point_of_interconnection')

# ---------------------- Data Preparation ---------------------- #

# Ensure necessary columns exist in itemized_df
required_itemized_columns = ['q_id', 'type_of_upgrade', 'point_of_interconnection', 'estimated_cost_x_1000', 'escalated_cost_x_1000']
for col in required_itemized_columns:
    if col not in itemized_df.columns:
        print(f"Warning: '{col}' column is missing in the itemized dataset.")
        if col in ['q_id', 'type_of_upgrade', 'point_of_interconnection']:
            itemized_df[col] = 'UNKNOWN'
        else:
            itemized_df[col] = 0

# Ensure necessary columns exist in totals_df
required_totals_columns = ['q_id', 'type_of_upgrade', 'point_of_interconnection', TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]
for col in required_totals_columns:
    if col not in totals_df.columns:
        print(f"Error: '{col}' column is missing in the totals dataset. Cannot proceed.")
        exit(1)

# Convert cost columns to numeric, coercing errors to NaN and filling with 0
cost_columns_itemized = ['estimated_cost_x_1000', 'escalated_cost_x_1000']
for col in cost_columns_itemized:
    itemized_df[col] = pd.to_numeric(itemized_df[col], errors='coerce').fillna(0)

cost_columns_totals = [TOTALS_ESTIMATED_COLUMN, TOTALS_ESCALATED_COLUMN]
for col in cost_columns_totals:
    totals_df[col] = pd.to_numeric(totals_df[col], errors='coerce').fillna(0)



cost_columns_network = ['estimated_cost_x_1000', 'escalated_cost_x_1000']
for col in cost_columns_network:
    total_network_df[col] = pd.to_numeric(total_network_df[col], errors='coerce').fillna(0)





# ---------------------- Calculate Manual Totals ---------------------- #

# Group itemized data by q_id and type_of_upgrade and calculate sums
itemized_grouped = itemized_df.groupby(['q_id', 'type_of_upgrade']).agg({
    'estimated_cost_x_1000': 'sum',
    'escalated_cost_x_1000': 'sum'
}).reset_index()

# Apply preference: Use estimated_cost_x_1000 if sum > 0, else use escalated_cost_x_1000
itemized_grouped['manual_total'] = itemized_grouped.apply(
    lambda row: row['estimated_cost_x_1000'] if row['estimated_cost_x_1000'] > 0 else row['escalated_cost_x_1000'],
    axis=1
)

# ---------------------- Prepare Totals Data ---------------------- #

# Group totals data by q_id and type_of_upgrade and calculate sums
totals_grouped = totals_df.groupby(['q_id', 'type_of_upgrade']).agg({
    TOTALS_ESTIMATED_COLUMN: 'sum',
    TOTALS_ESCALATED_COLUMN: 'sum'
}).reset_index()

# Apply preference: Use estimated_cost_x_1000 if sum > 0, else use escalated_cost_x_1000
totals_grouped['reported_total'] = totals_grouped.apply(
    lambda row: row[TOTALS_ESTIMATED_COLUMN] if row[TOTALS_ESTIMATED_COLUMN] > 0 else row[TOTALS_ESCALATED_COLUMN],
    axis=1
)

# ---------------------- Merge Data ---------------------- #

# Merge the itemized and totals data on q_id and type_of_upgrade
comparison_df = pd.merge(
    itemized_grouped,
    totals_grouped[['q_id', 'type_of_upgrade', 'reported_total']],
    on=['q_id', 'type_of_upgrade'],
    how='left'
)

# ---------------------- Check for Missing Upgrades ---------------------- #

# Identify q_ids that are missing any of the required upgrades
missing_upgrades_report = []
for q_id in comparison_df['q_id'].unique():
    upgrades_present = comparison_df[comparison_df['q_id'] == q_id]['type_of_upgrade'].unique()
    missing_upgrades = [upgrade for upgrade in REQUIRED_UPGRADES if upgrade not in upgrades_present]
    if missing_upgrades:
        missing_upgrades_report.append((q_id, missing_upgrades))

# Report missing upgrades
if missing_upgrades_report:
    print("\nQ_ids with missing upgrades:")
    for q_id, missing in missing_upgrades_report:
        print(f"  Q_id {q_id} is missing upgrades: {', '.join(missing)}")
else:
    print("\nAll q_ids have all required upgrades.")


# ------------------------ Check for no duplicates in type of upgrade in total data ------------------------ #

 

# Identify duplicates by grouping by q_id and type_of_upgrade
duplicates = totals_df[totals_df.duplicated(subset=['q_id', 'type_of_upgrade'], keep=False)]

if not duplicates.empty:
    print("\nDuplicate upgrade types detected:")
    for q_id, group in duplicates.groupby('q_id'):
        upgrade_types = group['type_of_upgrade'].unique()
        print(f"  Q_id {q_id} has repeated upgrade types: {', '.join(upgrade_types)}")
else:
    print("\nNo type of upgrade is repeated for any Q_id.")        

# ---------------------- Compare Totals and Identify Mismatches ---------------------- #

# Initialize list to store mismatches
mismatches = []

# Iterate through each row to compare manual_total with reported_total
for index, row in comparison_df.iterrows():
    q_id = row['q_id']
    upgrade = row['type_of_upgrade']
    manual_total = row['manual_total']
    reported_total = row['reported_total']
    
    # Determine if both manual_total and reported_total are zero
    if manual_total == 0.0 and reported_total == 0.0:
        continue  # No mismatch
    # Determine if manual_total is zero and reported_total is missing or zero
    elif manual_total == 0.0 and (pd.isna(row['reported_total']) or reported_total == 0.0):
        continue  # No mismatch
    # If reported_total is missing (NaN) and manual_total is not zero
    elif pd.isna(row['reported_total']) and manual_total != 0.0:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: Missing")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': 'Missing'
        })
    # If manual_total is not zero and reported_total is zero
    elif manual_total != 0.0 and reported_total == 0.0:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: 0.0")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': reported_total
        })
    # If both totals are non-zero but differ beyond tolerance
    elif abs(manual_total - reported_total) > 1e+2:
        print(f"Mismatch: Q_id {q_id}, Upgrade '{upgrade}' - Manual Total: {manual_total}, Reported Total: {reported_total}")
        mismatches.append({
            'q_id': q_id,
            'type_of_upgrade': upgrade,
            'manual_total': manual_total,
            'reported_total': reported_total
        })
    # Else, totals match; do nothing

# Create a DataFrame for mismatches
mismatches_df = pd.DataFrame(mismatches, columns=['q_id', 'type_of_upgrade', 'manual_total', 'reported_total'])

# Save mismatches to a CSV file
try:
    mismatches_df.to_csv(MISMATCHES_CSV_PATH, index=False)
    print(f"\nMismatches saved to '{MISMATCHES_CSV_PATH}'.")
except Exception as e:
    print(f"Error saving mismatches CSV: {e}")



# ---------------------- Compare Total Network vs Totals Data ---------------------- #

# ---------------------- Compare Total Network vs Totals Data ---------------------- #

# Function to extract summed cost per `q_id` while prioritizing `estimated_cost_x_1000`
def get_total_cost(df, upgrade_types):
    cost_sums = df[df['type_of_upgrade'].isin(upgrade_types)].groupby('q_id').agg({
        'estimated_cost_x_1000': 'sum',
        'escalated_cost_x_1000': 'sum'
    }).reset_index()

    # Apply logic: Use `estimated_cost_x_1000`, fallback to `escalated_cost_x_1000` if 0
    cost_sums['total_cost'] = cost_sums.apply(
        lambda row: row['estimated_cost_x_1000'] if row['estimated_cost_x_1000'] > 0 else row['escalated_cost_x_1000'], 
        axis=1
    )

    return cost_sums.set_index('q_id')['total_cost']

# Extract ADNU from total_network_df
total_adnu_network = get_total_cost(total_network_df, ['ADNU'])

# Extract ADNU from totals_df
total_adnu_totals = get_total_cost(totals_df, ['ADNU'])

# Extract RNU, LDNU, PNU, OPNU from totals_df and sum them
total_rnu_related = get_total_cost(totals_df, ['RNU', 'LDNU', 'PNU', 'OPNU'])

# ---------------------- Loop Over Q_IDs and Extract Values ---------------------- #
print("\n🔍 **ADNU Comparison Results:**")
for q_id in total_adnu_totals.index:
    # Extract ADNU values from both datasets
    adnu_total_sum = total_adnu_totals.get(q_id, 0)

    # Explicitly fetch ADNU from total_network_df per q_id
    adnu_total_network = (
        total_network_df[(total_network_df['q_id'] == q_id) & 
                         (total_network_df['type_of_upgrade'].str.strip().str.upper() == 'ADNU')]
    )['estimated_cost_x_1000']

    # Ensure we extract a single value
    if not adnu_total_network.empty:
        adnu_total_network = adnu_total_network.iloc[0]  # Take first value if multiple exist
    else:
        adnu_total_network = 0  # Default to 0 if not found

    # Compare values
    if abs(adnu_total_network - adnu_total_sum) > 10:
        print(f"❌ Mismatch in ADNU for Q_ID {q_id}:")
        print(f"   - Total Network (ADNU): {adnu_total_network}")
        print(f"   - Totals Dataset (ADNU): {adnu_total_sum}")
        print(f"   - Difference: {abs(adnu_total_network - adnu_total_sum)}\n")
    else:
        print(f"✅ Match for Q_ID {q_id}: {adnu_total_network} (Total Network) == {adnu_total_sum} (Totals)")

# ---------------------- Compare Summed RNU-related Against Total Network RNU ---------------------- #
print("\n🔍 **RNU Comparison Results:**")
for q_id in total_rnu_related.index:
    rnu_total_sum = total_rnu_related.get(q_id, 0)

    # Explicitly fetch RNU from total_network_df per q_id
    rnu_total_network = (
        total_network_df[(total_network_df['q_id'] == q_id) & 
                         (total_network_df['type_of_upgrade'].str.strip().str.upper() == 'RNU')]
    )['estimated_cost_x_1000']

    # Ensure we extract a single value
    if not rnu_total_network.empty:
        rnu_total_network = rnu_total_network.iloc[0]  # Take first value if multiple exist
    else:
        rnu_total_network = 0  # Default to 0 if not found

    # Compare values
    if abs(rnu_total_network - rnu_total_sum) > 10:
        print(f"❌ Mismatch in RNU-related for Q_ID {q_id}:")
        print(f"   - Total Network (RNU): {rnu_total_network}")
        print(f"   - Sum of RNU, LDNU, PNU, OPNU from Totals: {rnu_total_sum}")
        print(f"   - Difference: {abs(rnu_total_network - rnu_total_sum)}\n")
    else:
        print(f"✅ Match for Q_ID {q_id}: {rnu_total_network} (Total Network) == {rnu_total_sum} (Totals)")

print("\n✅ ADNU and RNU mismatch checks completed.")





# ---------------------- Point of Interconnection Matching ---------------------- #

# Extract unique q_id and point_of_interconnection from itemized dataset
itemized_poi = itemized_df[['q_id', 'point_of_interconnection']].drop_duplicates()

# Extract unique q_id and point_of_interconnection from totals dataset
totals_poi = totals_df[['q_id', 'point_of_interconnection']].drop_duplicates()

# Merge both to have a complete list of q_id and point_of_interconnection
all_poi = pd.concat([itemized_poi, totals_poi]).drop_duplicates().reset_index(drop=True)

# ---------------------- Direct Match Identification ---------------------- #

# Group by point_of_interconnection to find q_ids sharing the same point_of_interconnection
direct_matches = all_poi.groupby('point_of_interconnection')['q_id'].apply(list).reset_index()

# Filter groups with more than one q_id (i.e., shared points_of_interconnection)
direct_matches = direct_matches[direct_matches['q_id'].apply(len) > 1]

print("\nDirect Matches (Exact Point of Interconnection Names):")
if not direct_matches.empty:
    print(direct_matches)
else:
    print("No direct matches found.")

# ---------------------- Fuzzy Match Identification ---------------------- #

# Prepare list of points_of_interconnection for fuzzy matching
poi_list = all_poi['point_of_interconnection'].unique().tolist()

# Initialize list to store fuzzy matches
fuzzy_matches = []

# Iterate through each point_of_interconnection to find similar ones
for i, poi in enumerate(poi_list):
    # Compare with the rest of the points to avoid redundant comparisons
    similar_pois = process.extract(poi, poi_list[i+1:], scorer=fuzz.token_set_ratio)
    
    # Filter matches with similarity >= 80%
    for match_poi, score in similar_pois:
        if score >= 80:
            # Retrieve q_ids for both points_of_interconnection
            qids_poi1 = all_poi[all_poi['point_of_interconnection'] == poi]['q_id'].tolist()
            qids_poi2 = all_poi[all_poi['point_of_interconnection'] == match_poi]['q_id'].tolist()
            
            # Append the matched pairs with their points_of_interconnection and similarity score
            fuzzy_matches.append({
                'point_of_interconnection_1': poi,
                'q_ids_1': qids_poi1,
                'point_of_interconnection_2': match_poi,
                'q_ids_2': qids_poi2,
                'similarity_score': score
            })

# Convert fuzzy matches to DataFrame
fuzzy_matches_df = pd.DataFrame(fuzzy_matches)

print("\nFuzzy Matches (>=80% Similarity in Point of Interconnection):")
if not fuzzy_matches_df.empty:
    print(fuzzy_matches_df)
else:
    print("No fuzzy matches found.")

# ---------------------- Save Matched Q_ids to CSV ---------------------- #

# For clarity, create a combined DataFrame for direct and fuzzy matches

# Direct matches: list each pair of q_ids sharing the same point_of_interconnection
direct_matches_expanded = []
for _, row in direct_matches.iterrows():
    qids = row['q_id']
    poi = row['point_of_interconnection']
    # Generate all possible unique pairs
    for i in range(len(qids)):
        for j in range(i+1, len(qids)):
            direct_matches_expanded.append({
                'match_type': 'Direct',
                'point_of_interconnection_1': poi,
                'q_id_1': qids[i],
                'point_of_interconnection_2': poi,
                'q_id_2': qids[j],
                'similarity_score': 100
            })

# Fuzzy matches: already have pairs
fuzzy_matches_expanded = []
for _, row in fuzzy_matches_df.iterrows():
    fuzzy_matches_expanded.append({
        'match_type': 'Fuzzy',
        'point_of_interconnection_1': row['point_of_interconnection_1'],
        'q_id_1': row['q_ids_1'],
        'point_of_interconnection_2': row['point_of_interconnection_2'],
        'q_id_2': row['q_ids_2'],
        'similarity_score': row['similarity_score']
    })

# Convert to DataFrame
matched_qids_df = pd.DataFrame(direct_matches_expanded + fuzzy_matches_expanded)

# Save matched q_ids to CSV
try:
    matched_qids_df.to_csv(MATCHED_QIDS_CSV_PATH, index=False)
    print(f"Matched Q_ids saved to '{MATCHED_QIDS_CSV_PATH}'.")
except Exception as e:
    print(f"Error saving matched Q_ids CSV: {e}")

# ---------------------- Summary ---------------------- #

# Print a summary
total_checked = len(comparison_df)
total_mismatches = len(mismatches_df)
print(f"\nTotal checks performed: {total_checked}")
print(f"Total mismatches found: {total_mismatches}")


Loaded itemized data from /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_itemized.csv
Loaded totals data from /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_total.csv
Loaded total network data from /Users/vk365/Dropbox/Interconnections_data/data/pdf_scraper/output/Cluster 5/02_intermediate/costs_phase_1_cluster_5_style_D_total_network.csv

Q_ids with missing upgrades:
  Q_id 877 is missing upgrades: LDNU, CANU, OPNU
  Q_id 885 is missing upgrades: RNU, LDNU, CANU, OPNU
  Q_id 894 is missing upgrades: CANU, OPNU
  Q_id 898 is missing upgrades: LDNU, CANU, OPNU
  Q_id 899 is missing upgrades: LDNU, CANU, OPNU
  Q_id 900 is missing upgrades: RNU, CANU, OPNU
  Q_id 901 is missing upgrades: RNU, CANU, OPNU
  Q_id 910 is missing upgrades: PTO_IF, LDNU, CANU, OPNU
  Q_id 916 is missing upgrades: CANU, OPNU
  Q_id 917 is missing upgrades: C